In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("data3.csv")

In [10]:
data

,xAcc,yAcc,zAcc,xGyro,yGyro,zGyro,label
0,6.99,-0.57,-7.28,-2.75,-3.23,2.62,fall
1,6.51,-0.75,-6.93,-0.67,-6.35,4.64,fall
2,6.22,-0.63,-6.73,0.79,-5.49,3.85,fall
3,6.34,-0.62,-6.80,1.59,-2.26,0.67,fall
4,6.49,-0.39,-6.60,0.67,-0.24,-1.10,fall
...,...,...,...,...,...,...,...
96795,7.30,-1.16,-4.58,18.19,3.60,-35.77,light
96796,7.39,-0.37,-5.05,20.08,0.06,-34.55,light
96797,7.52,-1.46,-5.82,22.58,0.12,-28.02,light
96798,7.58,-2.14,-6.31,19.47,2.44,-22.52,light


### Adjust the features as we want

In [7]:
# every row is an observation of length 400 x 3 x 2

num_rows = data.shape[0]
# Set the step size to select every 400 rows
step = 400

# Create a range of indices to select rows in chunks of 400
indices = np.arange(0, num_rows, step)

# Create a list to store the transposed data
transposed_data = []
new_data = pd.DataFrame()
# Iterate over the original columns
for col in data.columns:
    if col != "label":
        # Extract values from the column and reshape into chunks of 400 rows
        column_values = data[col].values
        # reshaped_values = column_values[indices]
        # Split the list into chunks of 400 values
        chunk_size = 400
        chunks = [column_values[i:i + chunk_size] for i in range(0, len(column_values), chunk_size)]

        # Create a DataFrame
        tmp = pd.DataFrame(chunks, columns=[f'{col}_{i+1}' for i in range(chunk_size)])
        new_data = pd.concat([new_data, tmp], axis=1)

labels = np.array(data.label.iloc[np.arange(0,data.shape[0], 400)])
new_data["label"] = labels

In [103]:
# TEST PER LE SOMME DI VETTORI

# x = np.array(new_data.iloc[0, 1200:1600])
# y = np.array(new_data.iloc[0, 1600:2000])
# z = np.array(new_data.iloc[0, 2000:2400])
# s = x**2 + y**2 + z**2
# s

array([24.8598, 62.301, 45.58670000000001, 8.084599999999998,
       1.7165000000000004, 10.245700000000001, 25.401000000000003,
       57.993399999999994, 36.3779, 13.777400000000002, 35.7806, 85.7354,
       218.48379999999997, 392.6813, 611.6712, 678.1588999999999,
       555.9916000000001, 234.18890000000002, 33.5285, 71.1542, 155.1843,
       132.763, 65.9826, 26.888299999999997, 11.6146, 37.8709, 161.139,
       401.4936, 492.413, 378.2651, 243.3157, 276.9077,
       467.75149999999996, 666.1621, 915.46, 1222.4053000000001,
       1461.3521, 1524.3140999999996, 1421.9522000000002, 1042.4374,
       322.895, 368.60420000000005, 687.6625, 590.4003, 2163.8058,
       4108.514999999999, 2656.7885, 1091.8058, 395.2494000000001,
       772.4921, 1835.0937000000001, 2419.7801000000004, 2484.9901,
       2427.5726000000004, 1747.1200999999999, 1225.6753999999999,
       1157.6475, 2303.0545, 3530.652, 1358.7889, 1314.5949, 2128.7052,
       1546.0115999999998, 733.1405000000001, 236.1301

In [8]:
# every row is an observation of length 400 x 3 x 2

num_rows = new_data.shape[0]
# Set the step size to select every 400 rows
step = 400

# Create a range of indices to select rows in chunks of 400
indices = np.arange(0, num_rows, step)

# Create a list to store the transposed data
new_data2 = pd.DataFrame()
# Iterate over the original columns

def chunk_splitting(row):

    # Split the list into chunks of 400 values
    n_chunks = 2400/400
    chunks = np.array_split(np.array(row.values[:len(row.values)-1], dtype=float), n_chunks)

    return chunks

def acc_sum(vec):
    acc = np.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
    
    return acc

def gyr_sum(vec):
    gyr = np.sqrt(vec[3]**2 + vec[4]**2 + vec[5]**2)

    return gyr

chunks = new_data.apply(lambda x: chunk_splitting(x), axis=1)

In [9]:
df = pd.DataFrame()

new_data["acc_sum"] = chunks.apply(lambda x: acc_sum(x))
new_data["gyr_sum"] = chunks.apply(lambda x: gyr_sum(x))
df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
df[[f'gyr_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.gyr_sum.to_list(), index = new_data.index)
df

/tmp/ipykernel_14259/421098076.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
/tmp/ipykernel_14259/421098076.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
/tmp/ipykernel_14259/421098076.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7,acc_8,acc_9,acc_10,...,gyr_391,gyr_392,gyr_393,gyr_394,gyr_395,gyr_396,gyr_397,gyr_398,gyr_399,gyr_400
0,10.108581,9.537688,9.185761,9.317725,9.264567,9.625679,9.470449,9.866560,9.931556,9.772226,...,1.029612,0.832226,1.108152,1.508940,1.350296,1.384558,1.316586,1.062873,0.996393,1.218113
1,9.642505,9.398622,9.307653,9.317312,9.578157,9.677939,9.672518,9.602609,9.526610,9.640954,...,1.037593,0.755910,1.080139,1.166619,1.393198,1.229349,1.508575,1.302728,1.196746,1.042785
2,9.661987,9.639725,9.577270,9.591632,9.661698,9.597072,9.501800,9.647844,9.755275,9.684782,...,44.488846,36.741091,35.978113,39.397108,45.323665,50.354353,51.365726,49.815388,50.785400,42.545700
3,9.480295,8.876773,8.682062,8.864181,8.687900,8.705142,9.638350,17.116331,10.001335,8.002706,...,8.866268,5.678010,9.178181,9.709485,7.585269,6.007537,6.188667,5.522871,6.093423,10.975719
4,9.567241,9.223112,9.656159,9.535397,9.125322,8.799966,8.890231,9.637935,15.429051,11.867860,...,25.833149,25.946726,28.139966,31.458948,28.856559,20.350327,9.901773,10.485676,20.769095,45.044041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,9.170093,9.273748,9.343463,9.500316,9.085670,9.075230,9.275139,11.338188,12.609592,10.298942,...,12.656465,24.157347,14.040239,4.396464,12.434846,38.257195,49.500611,52.183440,56.153815,51.446970
238,9.022505,9.271451,9.181166,9.263369,9.303322,9.903297,11.889706,12.140906,10.008227,8.930291,...,11.848932,16.680126,22.309641,26.740989,36.154228,52.353129,67.389355,65.728835,38.954941,21.060769
239,7.912073,9.007675,8.720384,10.116961,9.617723,10.637171,10.104375,9.309882,9.280781,9.631895,...,24.538594,27.804604,30.426965,30.604902,30.377567,24.071066,8.991780,5.798396,17.283695,27.881836
240,9.129847,8.806730,9.078882,10.509943,10.325803,9.800066,9.796203,9.374711,9.540325,9.469789,...,12.731873,14.292617,14.283893,12.658535,6.578982,7.462721,15.174574,18.183570,21.144408,29.404588


In [12]:
# little check
np.sqrt([6.99**2 + (-0.57)**2 + (-7.28)**2])

array([10.10858051])

### Feature extraction

In [14]:
import re
from typing import *
from scipy.fft import fft, fftfreq
from scipy.signal import periodogram
import statsmodels.api as sm
from scipy.stats import skew
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression


def maxbin(row: Union[List[float], np.ndarray], plot: bool, n_bins: int = 10) -> List[float]:
    """
    Divide the data into a specified number of bins and calculate the maximum value for each bin.

    Args:
        row (list or array): The data to be divided into bins.
        num_bins (int, optional): The number of bins to divide the data into. Defaults to 10.

    Returns:
        list: A list containing the maximum value for each bin.
    """
    n = len(row)
    freq = list(fftfreq(n*2))
    step = n//n_bins
    max_values = []
    max_freqs = []
    for i in range(0, n+1, step):
        if i==0:
            prec = i
            continue
        mm = max(row[prec: i])
        max_values.append(mm)
        max_freqs.append(freq[list(row).index(mm)])
        prec = i
    if plot:
        return max_freqs, max_values
    else:
        return max_values


def fourier_magnitudes(signal: np.ndarray, n_bins: int = 10, plot: bool = False) -> np.ndarray:
    """
    This function takes a matrix (signal) which contains a time series for each row.
    It applies the Fast Fourier Transform and uses the get_max_per_bin function to find the maximum peaks for each bin of the arrays of the power spectrum for the FFT.

    Args:
        signal (np.ndarray): The input matrix, where each row represents a time series.
        num_bins (int, optional): The number of bins to divide the data into. Defaults to 10.

    Returns:
        np.ndarray: The maximum peaks for each bin of the arrays of the power spectrum for the FFT.
    """
    signal= np.array(signal)
    # let's calculate the FFT
    magnitudes = np.apply_along_axis(fft, 0, signal) 
    # extract just the first half of power spectrum and discover the peaks
    if len(magnitudes.shape)==1:
        magnitudes=magnitudes[:((magnitudes.shape[0])//2)]
    else:
        magnitudes=magnitudes[:((magnitudes.shape[0])//2),:]
    # apply power spectrum formula
    magnitudes = np.abs(magnitudes)**2 
    # take the peaks for each bin
    peaks = np.apply_along_axis(lambda x: maxbin(x, plot, n_bins), 0, magnitudes)
    
    return peaks


def psd_stats(signal: np.ndarray) -> np.ndarray:
    """
    This function takes a matrix (signal) which contains a time series for each row.
    It calculates the Power Spectral Density (PSD) from the signal and returns three different summary statistics
    from the distribution of PSD: the median, the mean absolute deviation, and the skewness (third moment).

    Args:
        signal (np.ndarray): The input matrix, where each row represents a time series.

    Returns:
        np.ndarray: An array containing the median, mean absolute deviation, and skewness of the Power Spectral Density.
    """
    signal= np.array(signal)
    # let's calculate the Power Spectral Density
    psd = np.array(np.abs(np.apply_along_axis(periodogram, 0, signal)[1]), dtype=np.float64)
    
    # extract our main summaries from the distribution
    median = np.median(psd, axis=0)
    # calculate the third moment (skewness)
    third_moment = skew(psd, axis=0)
    # calculate the mean absolute deviation
    mad = np.mean(np.abs(psd - np.mean(psd, axis=0)), axis = 0)
    
    return np.array([median, mad, third_moment])


def acf(signal: np.ndarray, n_lags: int = 10) -> np.ndarray:
    """
    This function takes a matrix (signal) which contains a time series for each row.
    It calculates the auto-correlation for different lags.

    Args:
        signal (np.ndarray): The input matrix, where each row represents a time series.
        num_lags (int, optional): The number of lags to calculate the auto-correlation for. Defaults to 10.

    Returns:
        np.ndarray: The auto-correlation for each time series in the input matrix.
    """
    signal= np.array(signal)
    autocorrelations = np.apply_along_axis(lambda x: sm.tsa.acf(x, nlags=n_lags), 0, signal)

    return autocorrelations


def adjust_df(s: pd.Series) -> pd.DataFrame:
    """
    This function takes a pandas Series, explodes and transposes it, and returns it as a DataFrame.

    Args:
        series (pd.Series): The input pandas Series.

    Returns:
        pd.DataFrame: The adjusted DataFrame.
    """
    df_list = [pd.DataFrame(matrix, index=[index]*len(matrix)) for index, matrix in s.items()]
    new_df = pd.concat(df_list, axis=0)
    new_df.columns = range(new_df.shape[1])
    new_df = new_df.T
    return new_df


def get_season(date):
    if date.month in [3, 4, 5]:
        return 1 # spring
    elif date.month in [6, 7, 8]:
        return 2 # summer
    elif date.month in [9, 10, 11]:
        return 3 # autumn
    else:
        return 4 # winter
    

def preproc(df, n_bins=10, n_lags=10):
    
    # group timeseries by accelerometer (device)
    group_dict = {}
    for col in df.columns:
        match = re.match("^([A-Z1-9]{4})", col)
        group_dict[col] = match.group() if match else None

    new_df = df.T.groupby(group_dict, axis=0)

    # evaluate max magnitudes (intesities of the frequencies from Fourier Fast transform) for each observation for each accelerometer
    magns = new_df.apply(lambda x: fourier_magnitudes(x, n_bins))

    # change col names of our dataframe of magnitudes
    existing_columns = magns.columns
    # generate a list of new column names
    new_columns = []
    i=1
    for j,col_name in enumerate(existing_columns):
        if ((j+1)%(n_bins+1)) == 0:
            i=1
        new_columns.append(f"{col_name}_mag_max_{i}")
        i+=1
    magns.columns = new_columns

    new_df = adjust_df(magns)
    return new_df

In [21]:
prova = preproc(new_data)

C:\Users\engri\AppData\Local\Temp\ipykernel_5748\310717205.py:152: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  magns = new_df.apply(lambda x: fourier_magnitudes(x, n_bins))


In [22]:
prova

""
0
1
2
3
4
...
237
238
239
240
